<a href="https://colab.research.google.com/github/Deepakshi-Singhla/Codsoft/blob/main/CreditCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CreditCard Fraud Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('fraudTrain.csv', index_col=0) #By passing index 0, first column becomes the index column
df

C:\Users\DELL\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,Hatch,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,Tuscarora,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,Manderson,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


In [2]:
df1 = df['is_fraud'].value_counts()
df1 #data is imbalanced

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [3]:
legit = df[df['is_fraud'] == 0]
fraud = df[df['is_fraud'] == 1]


In [4]:
#To balance the data
legit_sample = legit.sample(n=7506)
df = pd.concat([legit_sample, fraud], axis = 0)
df.shape

(15012, 22)

In [5]:
outliear = len(fraud)/float(len(legit))
print(outliear)

0.005822355331224998


In [6]:
df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.501200e+04,15012.000000,15012.000000,15012.000000,15012.000000,1.501200e+04,1.501200e+04,15012.000000,15012.000000,15012.000000
mean,3.998996e+17,298.836577,48609.284106,38.606811,-90.152466,9.475299e+04,1.348887e+09,38.602626,-90.151472,0.500000
std,1.279717e+18,368.863988,27060.513092,5.119888,14.000387,3.188138e+05,1.329832e+07,5.161944,14.021490,0.500017
min,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.325379e+09,19.275449,-166.550779,0.000000
25%,1.800401e+14,20.077500,25832.000000,34.882900,-96.809400,7.370000e+02,1.337554e+09,34.947455,-96.886487,0.000000
50%,3.521815e+15,89.020000,47842.000000,39.405500,-87.043600,2.523000e+03,1.349034e+09,39.422297,-87.228171,0.500000
75%,4.635331e+15,479.470000,71960.000000,41.894800,-80.138100,2.113400e+04,1.359768e+09,41.951114,-80.196969,1.000000
max,4.992346e+18,2398.970000,99783.000000,66.693300,-67.950300,2.906700e+06,1.371804e+09,67.510267,-66.971614,1.000000


In [7]:
df_R = df[['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time' , 'merch_lat','merch_long', 'is_fraud']]
df_R.shape

(15012, 10)

In [8]:
df_R['is_fraud'].value_counts()

0    7506
1    7506
Name: is_fraud, dtype: int64

In [9]:
X = df_R.drop('is_fraud', axis=1)
Y = df_R['is_fraud']
X.shape
Y.shape

(15012,)

In [10]:
xdata = X.values

ytarget = Y.values.ravel()


In [11]:
xTrain, xTest, yTrain, yTest = train_test_split(xdata, ytarget, test_size=0.2, random_state= 42)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [13]:
rfc.fit(xTrain,yTrain)

RandomForestClassifier()

In [14]:
ypred1 = rfc.predict(xTest)

In [15]:
acc = accuracy_score(yTest, ypred1)
print(acc)

0.9134199134199135


In [16]:
#accuracy on test data
X_test_prediction_R = rfc.predict(xTest)
test_data_accuracy  = accuracy_score(X_test_prediction_R, yTest)
print(test_data_accuracy)

0.9134199134199135


Test Data


In [17]:
df_test_R = pd.read_csv('fraudTest.csv', index_col=0)
df_test_R

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [18]:
df_test_R = df_test_R[['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time' , 'merch_lat','merch_long', 'is_fraud']]

In [19]:
X_TEST_R = df_test_R.drop('is_fraud', axis=1)
Y_TEST_R = df_test_R['is_fraud']

In [20]:
print(X_TEST_R.shape)
print(Y_TEST_R.shape)

(555719, 9)
(555719,)


In [21]:
Y_TEST_PRED_R = rfc.predict(X_TEST_R)
final_test1 =  accuracy_score(Y_TEST_PRED_R, Y_TEST_R)
print(final_test1)

0.8747748412417067
